<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/AllCF_Sent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import  drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
nltk.download('all')
from nltk.corpus import sentiwordnet as swn
device_name = tf.test.gpu_device_name()
print(device_name)
df = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/crowdFlower/crowdFlower_clean.csv')
df = df.sample(frac=1,random_state=32)

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

In [ ]:
df.head()

,Unnamed: 0,emotion,tweets
35975,35975,6,katofawesome praying for love in a lap dance a...
17590,17590,1,trying to fix my internet connectionguess my p...
38071,38071,0,starting an account here on twitter
23304,23304,1,dipfico hmm wrong link ignore my tweet
26990,26990,0,mzunyque thanks before the major chop


In [ ]:
df.emotion.unique()

array([ 6,  1,  0,  2,  3,  4, 10,  5,  7,  8,  9, 12, 11])

In [ ]:
category_dict = {0:[1,0,0,0,0,0,0,0,0,0,0,0,0],1:[0,1,0,0,0,0,0,0,0,0,0,0,0],2:[0,0,1,0,0,0,0,0,0,0,0,0,0],3:[0,0,0,1,0,0,0,0,0,0,0,0,0],4:[0,0,0,0,1,0,0,0,0,0,0,0,0],5:[0,0,0,0,0,1,0,0,0,0,0,0,0],6:[0,0,0,0,0,0,1,0,0,0,0,0,0],7:[0,0,0,0,0,0,0,1,0,0,0,0,0],8:[0,0,0,0,0,0,0,0,1,0,0,0,0],9:[0,0,0,0,0,0,0,0,0,1,0,0,0],10:[0,0,0,0,0,0,0,0,0,0,1,0,0],11:[0,0,0,0,0,0,0,0,0,0,0,1,0],12:[0,0,0,0,0,0,0,0,0,0,0,0,1]}
data_tweet = [x.lower().split() for x in df['tweets']]
data_cat = np.array([category_dict[x] for x in df['emotion']])

In [ ]:
print(data_tweet[:5])
print(data_cat[:5])

[['katofawesome', 'praying', 'for', 'love', 'in', 'a', 'lap', 'dance', 'and', 'paying', 'in', 'naivety', 'lt--one', 'of', 'my', 'fav', 'songs'], ['trying', 'to', 'fix', 'my', 'internet', 'connectionguess', 'my', 'prayers', 'have', 'been', 'answered', 'and', 'i', 'wont', 'have', 'any', 'study', 'distractions', 'ugh'], ['starting', 'an', 'account', 'here', 'on', 'twitter'], ['dipfico', 'hmm', 'wrong', 'link', 'ignore', 'my', 'tweet'], ['mzunyque', 'thanks', 'before', 'the', 'major', 'chop']]
[[0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0]]


In [ ]:
# Parameters
Min_count = 0
Embedding_size = 200
Window_size = 5
Negative_sampling = 00

In [ ]:
w2v_sg = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=1)
w2v_sg.build_vocab(data_tweet)
w2v_sg.train(data_tweet, total_examples=w2v_sg.corpus_count, epochs=5)
w2v_cbow = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=0)
w2v_cbow.build_vocab(data_tweet)
w2v_cbow.train(data_tweet, total_examples=w2v_cbow.corpus_count, epochs=5)
w2v_sg.wv.init_sims(True)
w2v_cbow.wv.init_sims(True)

In [ ]:
vocab_sg = w2v_sg.wv.vocab
vocab_sg = [x for x in vocab_sg]
vocab_cbow = w2v_cbow.wv.vocab
vocab_cbow = [x for x in vocab_cbow] 
vocab_glove = {}
with open("/content/drive/My Drive/InfluenceAnalysis/glove/glove.twitter.27B.200d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      vocab_glove[word] = vector

In [ ]:
batch_size = 100
Max_input_size = max([len(x) for x in data_tweet])
data_tweet = [nltk.pos_tag(x) for x in data_tweet]
def getsent(word,tag):
  res=0
  try:
    x = swn.senti_synset(word+'.'+tag[0].lower()+'.01')
    res =  (x.pos_score()-x.neg_score())
  finally:
    return res 
data_tweet = [[(i[0],i[1],getsent(i[0],i[1])) for i in x] for x in data_tweet]

In [ ]:
def vec_gen(w2v,vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor=2):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(w2v.wv.word_vec(i[0]),i[1],i[2]) for i in x if i[0] in vocab]
        l=len(tweet)
        for i in range(l):
          if tweet[i][2]<0:
            tweet[i]=((tweet[i][0]*(-1)),tweet[i][1],tweet[i][2])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)          
      
def glove_gen(vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor=2):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(vocab[i[0]],i[1],i[2]) for i in x if i[0] in vocab.keys()]
        l=len(tweet)
        for i in range(l):
          if tweet[i][2]<0:
            tweet[i]=((tweet[i][0]*(-1)),tweet[i][1],tweet[i][2])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_tweet, data_cat, test_size=0.20, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=42)
result_table = [0,0,0,0,0,0,0,0,0]

In [ ]:
epochs = 40

In [ ]:
model_sg = tf.keras.Sequential()
model_sg.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg.add(tf.keras.layers.LSTM(50))
model_sg.add(tf.keras.layers.Dense(13, activation='softmax'))
model_sg.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
sg_train_gen=vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size)
sg_val_gen=vec_gen(w2v_sg,vocab_sg,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_sg_lstm = model_sg.fit_generator(sg_train_gen,validation_data=sg_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_sg.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[0]=history[1]
print("SG_LSTM")


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
257/257 [==============================] - 118s 453ms/step - loss: 2.2601 - accuracy: 0.2095 - val_loss: 2.1384 - val_accuracy: 0.2174
Epoch 2/40
257/257 [==============================] - 114s 444ms/step - loss: 2.1469 - accuracy: 0.2161 - val_loss: 2.1360 - val_accuracy: 0.2175
Epoch 3/40
257/257 [==============================] - 112s 438ms/step - loss: 2.1479 - accuracy: 0.2187 - val_loss: 2.1367 - val_accuracy: 0.2180
Epoch 4/40
257/257 [==============================] - 112s 438ms/step - loss: 2.1459 - accuracy: 0.2211 - val_loss: 2.1260 - val_accuracy: 0.2558
Epoch 5/40
257/257 [==============================] - 113s 439ms/step - loss: 2.1266 - accuracy: 0.2548 - val_loss: 2.0985 - val_accuracy: 0.2649
Epoch 6/40
257/257 [==============================] - 113s 443ms/step - loss: 2.0938 - accuracy: 0.2722 - val_loss: 2.0604 - val_accuracy: 0.2792
Epoch 7/40
257/257 [==============================] - 115s 451ms/step - loss: 2.0605 - accuracy: 0.2876 - val_loss: 2.0505 -

In [ ]:
model_sg_bi = tf.keras.Sequential()
model_sg_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_sg_bi.add(tf.keras.layers.Dense(13, activation='softmax'))
model_sg_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_sg_bi = model_sg_bi.fit_generator(sg_train_gen,validation_data=sg_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_sg_bi.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[1]=history[1]
print("SG_BI")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
257/257 [==============================] - 130s 493ms/step - loss: 2.2416 - accuracy: 0.2132 - val_loss: 2.0895 - val_accuracy: 0.2643
Epoch 2/40
257/257 [==============================] - 126s 493ms/step - loss: 2.0755 - accuracy: 0.2756 - val_loss: 2.0551 - val_accuracy: 0.2858
Epoch 3/40
257/257 [==============================] - 123s 479ms/step - loss: 2.0337 - accuracy: 0.3057 - val_loss: 2.0383 - val_accuracy: 0.3026
Epoch 4/40
257/257 [==============================] - 124s 482ms/step - loss: 2.0152 - accuracy: 0.3098 - val_loss: 2.0256 - val_accuracy: 0.3071
Epoch 5/40
257/257 [==============================] - 128s 499ms/step - loss: 1.9969 - accuracy: 0.3155 - val_loss: 2.0223 - val_accuracy: 0.3088
Epoch 6/40
257/257 [==============================] - 127s 494ms/step - loss: 1.9812 - accuracy: 0.3206 - val_loss: 2.0158 - val_accuracy: 0.3100
Epoch 7/40
257/257 [==============================] - 124s 483ms/step - loss: 1.9607 - accuracy: 0.3297 - val_loss: 2.0067 -

In [ ]:
model_sg_rnn = tf.keras.Sequential()
model_sg_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_rnn.add(tf.keras.layers.SimpleRNN(50))
model_sg_rnn.add(tf.keras.layers.Dense(13, activation='softmax'))
model_sg_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_sg_rnn = model_sg_rnn.fit_generator(sg_train_gen,validation_data=sg_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_sg_rnn.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)


result_table[2]=history[1]
print("SG_RNN")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
257/257 [==============================] - 112s 432ms/step - loss: 2.2344 - accuracy: 0.2012 - val_loss: 2.1419 - val_accuracy: 0.2157
Epoch 2/40
257/257 [==============================] - 110s 430ms/step - loss: 2.1430 - accuracy: 0.2151 - val_loss: 2.1384 - val_accuracy: 0.2154
Epoch 3/40
257/257 [==============================] - 110s 428ms/step - loss: 2.1438 - accuracy: 0.2147 - val_loss: 2.1387 - val_accuracy: 0.2158
Epoch 4/40
257/257 [==============================] - 109s 424ms/step - loss: 2.1441 - accuracy: 0.2173 - val_loss: 2.1399 - val_accuracy: 0.2155
Epoch 5/40
257/257 [==============================] - 113s 443ms/step - loss: 2.1406 - accuracy: 0.2195 - val_loss: 2.1391 - val_accuracy: 0.2125
Epoch 6/40
257/257 [==============================] - 113s 440ms/step - loss: 2.1382 - accuracy: 0.2229 - val_loss: 2.1348 - val_accuracy: 0.2335
Epoch 7/40
257/257 [==============================] - 111s 435ms/step - loss: 2.1287 - accuracy: 0.2302 - val_loss: 2.1350 -

In [ ]:
model_cbow = tf.keras.Sequential()
model_cbow.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow.add(tf.keras.layers.LSTM(50))
model_cbow.add(tf.keras.layers.Dense(13, activation='softmax'))
model_cbow.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
cbow_train_gen=vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size)
cbow_val_gen=vec_gen(w2v_cbow,vocab_cbow,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_cbow_lstm = model_cbow.fit_generator(cbow_train_gen,validation_data=cbow_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_cbow.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[3]=history[1]
print("CBOW_LSTM")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
257/257 [==============================] - 119s 458ms/step - loss: 2.2606 - accuracy: 0.2135 - val_loss: 2.1229 - val_accuracy: 0.2563
Epoch 2/40
257/257 [==============================] - 114s 444ms/step - loss: 2.1198 - accuracy: 0.2456 - val_loss: 2.0792 - val_accuracy: 0.2611
Epoch 3/40
257/257 [==============================] - 113s 443ms/step - loss: 2.0794 - accuracy: 0.2687 - val_loss: 2.0611 - val_accuracy: 0.2808
Epoch 4/40
257/257 [==============================] - 112s 438ms/step - loss: 2.0495 - accuracy: 0.2905 - val_loss: 2.0417 - val_accuracy: 0.2963
Epoch 5/40
257/257 [==============================] - 114s 446ms/step - loss: 2.0262 - accuracy: 0.3072 - val_loss: 2.0341 - val_accuracy: 0.2971
Epoch 6/40
257/257 [==============================] - 113s 442ms/step - loss: 2.0139 - accuracy: 0.3139 - val_loss: 2.0232 - val_accuracy: 0.3011
Epoch 7/40
257/257 [==============================] - 114s 444ms/step - loss: 1.9995 - accuracy: 0.3232 - val_loss: 2.0199 -

In [ ]:
model_cbow_bi = tf.keras.Sequential()
model_cbow_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_cbow_bi.add(tf.keras.layers.Dense(13, activation='softmax'))
model_cbow_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_cbow_bi = model_cbow_bi.fit_generator(cbow_train_gen,validation_data=cbow_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_cbow_bi.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[4]=history[1]
print("CBOW_BI")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
257/257 [==============================] - 127s 481ms/step - loss: 2.2482 - accuracy: 0.2219 - val_loss: 2.0843 - val_accuracy: 0.2706
Epoch 2/40
257/257 [==============================] - 120s 468ms/step - loss: 2.0698 - accuracy: 0.2834 - val_loss: 2.0340 - val_accuracy: 0.2988
Epoch 3/40
257/257 [==============================] - 122s 477ms/step - loss: 2.0295 - accuracy: 0.3017 - val_loss: 2.0225 - val_accuracy: 0.3049
Epoch 4/40
257/257 [==============================] - 122s 475ms/step - loss: 2.0094 - accuracy: 0.3120 - val_loss: 2.0262 - val_accuracy: 0.3031
Epoch 5/40
257/257 [==============================] - 125s 487ms/step - loss: 1.9909 - accuracy: 0.3210 - val_loss: 2.0199 - val_accuracy: 0.3078
Epoch 6/40
257/257 [==============================] - 125s 487ms/step - loss: 1.9724 - accuracy: 0.3237 - val_loss: 2.0170 - val_accuracy: 0.3111
Epoch 7/40
257/257 [==============================] - 123s 480ms/step - loss: 1.9547 - accuracy: 0.3314 - val_loss: 2.0106 -

In [ ]:
model_cbow_rnn = tf.keras.Sequential()
model_cbow_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_rnn.add(tf.keras.layers.SimpleRNN(50))
model_cbow_rnn.add(tf.keras.layers.Dense(13, activation='softmax'))
model_cbow_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
cbow_train_gen=vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size)
cbow_val_gen=vec_gen(w2v_cbow,vocab_cbow,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_cbow_rnn = model_cbow_rnn.fit_generator(cbow_train_gen,validation_data=cbow_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_cbow_rnn.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[5]=history[1]
print("CBOW_RNN")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
257/257 [==============================] - 85s 329ms/step - loss: 2.2495 - accuracy: 0.1966 - val_loss: 2.1409 - val_accuracy: 0.2188
Epoch 2/40
257/257 [==============================] - 78s 306ms/step - loss: 2.1476 - accuracy: 0.2136 - val_loss: 2.1381 - val_accuracy: 0.2182
Epoch 3/40
257/257 [==============================] - 82s 322ms/step - loss: 2.1487 - accuracy: 0.2182 - val_loss: 2.1385 - val_accuracy: 0.2185
Epoch 4/40
257/257 [==============================] - 81s 318ms/step - loss: 2.1471 - accuracy: 0.2169 - val_loss: 2.1389 - val_accuracy: 0.2180
Epoch 5/40
257/257 [==============================] - 80s 310ms/step - loss: 2.1481 - accuracy: 0.2149 - val_loss: 2.1404 - val_accuracy: 0.2208
Epoch 6/40
257/257 [==============================] - 81s 315ms/step - loss: 2.1478 - accuracy: 0.2221 - val_loss: 2.1348 - val_accuracy: 0.2342
Epoch 7/40
257/257 [==============================] - 82s 321ms/step - loss: 2.1404 - accuracy: 0.2221 - val_loss: 2.1349 - val_ac

In [ ]:
model_glove = tf.keras.Sequential()
model_glove.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_glove.add(tf.keras.layers.LSTM(50))
model_glove.add(tf.keras.layers.Dense(13, activation='softmax'))
model_glove.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
glove_train_gen=glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size)
glove_val_gen=glove_gen(vocab_glove,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_glove_lstm = model_glove.fit_generator(glove_train_gen,validation_data=glove_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_glove.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[6]=history[1]
print("GLOVE_LSTM")


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
257/257 [==============================] - 19s 65ms/step - loss: 2.2313 - accuracy: 0.2329 - val_loss: 2.0119 - val_accuracy: 0.2937
Epoch 2/40
257/257 [==============================] - 16s 62ms/step - loss: 1.9883 - accuracy: 0.3079 - val_loss: 1.9293 - val_accuracy: 0.3334
Epoch 3/40
257/257 [==============================] - 16s 63ms/step - loss: 1.9153 - accuracy: 0.3393 - val_loss: 1.9032 - val_accuracy: 0.3440
Epoch 4/40
257/257 [==============================] - 16s 64ms/step - loss: 1.8743 - accuracy: 0.3565 - val_loss: 1.8859 - val_accuracy: 0.3494
Epoch 5/40
257/257 [==============================] - 16s 64ms/step - loss: 1.8481 - accuracy: 0.3665 - val_loss: 1.8841 - val_accuracy: 0.3475
Epoch 6/40
257/257 [==============================] - 16s 64ms/step - loss: 1.8313 - accuracy: 0.3714 - val_loss: 1.8804 - val_accuracy: 0.3495
Epoch 7/40
257/257 [==============================] - 16s 63ms/step - loss: 1.8057 - accuracy: 0.3796 - val_loss: 1.8777 - val_accuracy:

In [ ]:
model_glove_bi = tf.keras.Sequential()
model_glove_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_glove_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_glove_bi.add(tf.keras.layers.Dense(13, activation='softmax'))
model_glove_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_glove_bi = model_glove_bi.fit_generator(glove_train_gen,validation_data=glove_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_glove_bi.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[7]=history[1]
print("GLOVE_BI")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
257/257 [==============================] - 31s 108ms/step - loss: 2.1427 - accuracy: 0.2607 - val_loss: 1.9208 - val_accuracy: 0.3405
Epoch 2/40
257/257 [==============================] - 27s 104ms/step - loss: 1.8999 - accuracy: 0.3496 - val_loss: 1.8711 - val_accuracy: 0.3562
Epoch 3/40
257/257 [==============================] - 27s 107ms/step - loss: 1.8427 - accuracy: 0.3700 - val_loss: 1.8541 - val_accuracy: 0.3620
Epoch 4/40
257/257 [==============================] - 27s 105ms/step - loss: 1.8078 - accuracy: 0.3821 - val_loss: 1.8446 - val_accuracy: 0.3686
Epoch 5/40
257/257 [==============================] - 27s 105ms/step - loss: 1.7763 - accuracy: 0.3935 - val_loss: 1.8457 - val_accuracy: 0.3683
Epoch 6/40
257/257 [==============================] - 27s 106ms/step - loss: 1.7411 - accuracy: 0.4045 - val_loss: 1.8539 - val_accuracy: 0.3678
Epoch 7/40
257/257 [==============================] - 27s 105ms/step - loss: 1.7049 - accuracy: 0.4131 - val_loss: 1.8662 - val_ac

In [ ]:
model_glove_rnn = tf.keras.Sequential()
model_glove_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_glove_rnn.add(tf.keras.layers.SimpleRNN(50))
model_glove_rnn.add(tf.keras.layers.Dense(13, activation='softmax'))
model_glove_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_glove_rnn = model_glove_rnn.fit_generator(glove_train_gen,validation_data=glove_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_glove_rnn.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[8]=history[1]
print("GLOVE_RNN")

Epoch 1/40


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


257/257 [==============================] - 9s 32ms/step - loss: 2.2394 - accuracy: 0.2032 - val_loss: 2.1386 - val_accuracy: 0.2128
Epoch 2/40
257/257 [==============================] - 7s 28ms/step - loss: 2.1347 - accuracy: 0.2432 - val_loss: 2.1070 - val_accuracy: 0.2729
Epoch 3/40
257/257 [==============================] - 7s 29ms/step - loss: 2.1065 - accuracy: 0.2617 - val_loss: 2.0735 - val_accuracy: 0.2538
Epoch 4/40
257/257 [==============================] - 7s 26ms/step - loss: 2.0941 - accuracy: 0.2368 - val_loss: 2.1413 - val_accuracy: 0.2148
Epoch 5/40
257/257 [==============================] - 7s 26ms/step - loss: 2.1446 - accuracy: 0.2150 - val_loss: 2.1390 - val_accuracy: 0.2151
Epoch 6/40
257/257 [==============================] - 7s 26ms/step - loss: 2.1422 - accuracy: 0.2207 - val_loss: 2.1382 - val_accuracy: 0.2140
Epoch 7/40
257/257 [==============================] - 7s 26ms/step - loss: 2.1426 - accuracy: 0.2191 - val_loss: 2.1396 - val_accuracy: 0.2134
Epoch 8/40

In [20]:
print("\tLstm\t\t\tBiLstm\t\t\tRnn")
print("sg",result_table[:3])
print("cbow",result_table[3:6])
print("glove",result_table[6:9])
# print(history_sg_bi.history)
# plt.title("Skip-Gram + Bi-LSTM accuracy, loss vs epochs Graph")
# plt.plot(history_sg_bi.history['loss'],c='b',label='loss')
# plt.plot(history_sg_bi.history['accuracy'],c='r',label='accuracy')
# plt.xlabel('epochs')
# plt.legend()
# # plt.yticks()
# plt.show()
# plt.title("GloVe + Bi-LSTM accuracy, loss vs epochs Graph")
# plt.plot(history_glove_bi.history['loss'],c='b',label='loss')
# plt.plot(history_glove_bi.history['accuracy'],c='r',label='accuracy')
# plt.xlabel('epochs')
# plt.legend()
# # plt.yticks()
# plt.show()

	Lstm			BiLstm			Rnn
sg [0, 0, 0]
cbow [0, 0, 0.2434999942779541]
glove [0.30300000309944153, 0.2848750054836273, 0.2865000069141388]
